In [1]:
import anndata as ad
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scanpy as sc
import src.mvae.mt.mvae.utils as utils
import torch
import yaml

from scipy.io import mmread, mmwrite
from scipy.sparse import csr_matrix, save_npz, vstack
from src.lightning.gene import GeneModule
from src.mvae.mt.data import GeneDataset
from src.mvae.mt.mvae.distributions import *
from src.mvae.mt.mvae.models.gene_vae import GeneVAE
from src.mvae.mt.mvae.ops.hyperbolics import lorentz_to_poincare
from src.mvae.mt.mvae.ops.spherical import spherical_to_projected
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

In [ ]:
mtx_path = "/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/cellxgene_counts.mtx"
mtx = mmread(mtx_path).T.tocsr()
print(mtx.shape)

batch = pd.read_csv("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/cellxgene_batch.tsv", sep="\t", header=None)
print(batch.shape)

cell_type = pd.read_csv("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/cellxgene_cell_type.tsv", sep="\t", header=None)
print(cell_type.shape)

metadata = pd.read_csv("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/cellxgene_metadata.tsv", sep="\t")
print(metadata.shape)

cluster_assignments = np.load("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_cluster_assignments.npy")
print(cluster_assignments.shape)

In [ ]:
num_valid_indices = 25000
valid_indices = np.random.choice(np.arange(mtx.shape[0]), size=num_valid_indices, replace=False)
train_indices = np.setdiff1d(np.arange(mtx.shape[0]), valid_indices)
print(len(train_indices), len(valid_indices))
assert len(train_indices) == mtx.shape[0] - num_valid_indices
np.save("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_train_indices.npy", train_indices)
np.save("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_valid_indices.npy", valid_indices)

In [4]:
mtx_train = mtx[train_indices, :]
batch_train = batch.iloc[train_indices, :]
cell_type_train = cell_type.iloc[train_indices, :]
metadata_train = metadata.iloc[train_indices, :]
cluster_assignments_train = cluster_assignments[train_indices]

mmwrite("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_counts_train.mtx", mtx_train.T)
batch_train.to_csv("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_batch_train.tsv", sep="\t", index=False, header=False)
cell_type_train.to_csv("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_cell_type_train.tsv", sep="\t", index=False, header=False)
metadata_train.to_csv("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_metadata_train.tsv", sep="\t", index=False)
np.save("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_cluster_assignments_train.npy", cluster_assignments_train)

In [5]:
mtx_valid = mtx[valid_indices, :]
batch_valid = batch.iloc[valid_indices, :]
cell_type_valid = cell_type.iloc[valid_indices, :]
metadata_valid = metadata.iloc[valid_indices, :]
cluster_assignments_valid = cluster_assignments[valid_indices]

mmwrite("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_counts_valid.mtx", mtx_valid.T)
batch_valid.to_csv("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_batch_valid.tsv", sep="\t", index=False, header=False)
cell_type_valid.to_csv("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_cell_type_valid.tsv", sep="\t", index=False, header=False)
metadata_valid.to_csv("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_metadata_valid.tsv", sep="\t", index=False)
np.save("/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene/climb_cluster_assignments_valid.npy", cluster_assignments_valid)

In [ ]:
data_dir = "/home/romainlhardy/Downloads/merlin_cxg_2023_05_15_sf-log1p/test"
save_dir = "/home/romainlhardy/code/hyperbolic-cancer/data/cellxgene"
parquet_files = os.listdir(data_dir)
print(len(parquet_files))

In [ ]:
file_samples = 10
sampled_files = np.random.choice(parquet_files, size=file_samples)
highly_variable_mask = np.load(os.path.join(save_dir, "highly_variable_mask.npy"))
batch_columns = ["dataset_id", "donor_id"]
mtx = None
batch = None
cell_type = None

for i, file in enumerate(tqdm(sampled_files)):
    df = pd.read_parquet(os.path.join(data_dir, file), engine="pyarrow", use_threads=True)
    if not isinstance(df.X.iloc[0], np.ndarray):
        arrays = [np.array(x) for x in df.X]
    else:
        arrays = df.X.values

    try:
        X = np.stack(arrays, axis=0).astype(np.float32)
        X = np.expm1(X)[:, highly_variable_mask]
        X_sparse = csr_matrix(X)
    except ValueError as e:
        print(f"Error processing file {file} at index {i}: {e}")
        continue
    except TypeError as e:
         print(f"Error processing file {file} at index {i} (TypeError, possibly non-numeric data in X): {e}")
         continue

    b = df[batch_columns].values
    c = df["cell_type"].values

    mtx = X_sparse if mtx is None else vstack([mtx, X_sparse])
    batch = b if batch is None else np.vstack([batch, b])
    cell_type = c if cell_type is None else np.concatenate([cell_type, c])

if mtx is not None:
    print(f"Finished processing {len(sampled_files)} files. Saving final data...")
    try:
        mtx_path = os.path.join(save_dir, "climb_counts_test.mtx")
        batch_path = os.path.join(save_dir, "climb_batch_test.tsv")
        cell_type_path = os.path.join(save_dir, "climb_cell_type_test.tsv")

        mmwrite(mtx_path, mtx.T)
        np.savetxt(batch_path, batch, delimiter="\t")
        np.savetxt(cell_type_path, cell_type, delimiter="\t")
        print("Data saved successfully.")
    except Exception as e:
        print(f"Error saving final data: {e}")
else:
    print("No data processed or accumulated.")